In [1]:
import pandas as pd


# EDA

In [2]:

df = pd.read_csv(r"C:\Users\rajes\Datascience_jp\Day5\Supervised\diabetes1_unclean.csv")
df[13:18]


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
13,1,189,60,23.0,846,30.1,0.398,59,1.0
14,5,166,72,19.0,NaN,25.8,0.587,51,1.0
15,7,100,0,0.0,0,30.0,0.484,32,1.0
16,0,118,84,47.0,230,45.8,0.551,31,1.0
17,7,107,74,0.0,0,29.6,0.254,31,NaN


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             767 non-null    float64
 4   Insulin                   767 non-null    object 
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    object 
 7   Age                       768 non-null    int64  
 8   Outcome                   766 non-null    float64
dtypes: float64(3), int64(4), object(2)
memory usage: 54.1+ KB


In [4]:
df = df.apply(pd.to_numeric, errors='coerce')


In [5]:
df = df.ffill()
df[13:18]

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
13,1,189,60,23.0,846.0,30.1,0.398,59,1.0
14,5,166,72,19.0,846.0,25.8,0.587,51,1.0
15,7,100,0,0.0,0.0,30.0,0.484,32,1.0
16,0,118,84,47.0,230.0,45.8,0.551,31,1.0
17,7,107,74,0.0,0.0,29.6,0.254,31,1.0


In [6]:
df.duplicated().sum()
df['Outcome'] = df['Outcome'].apply(lambda x : 1 if x > 0 else 0)
df['Outcome']

0      1
1      0
2      1
3      0
4      1
      ..
763    0
764    0
765    0
766    1
767    0
Name: Outcome, Length: 768, dtype: int64

# Train the model

In [7]:
X = df.drop(columns = ['Outcome'])
y = df['Outcome']

In [8]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.ensemble import GradientBoostingClassifier



X_train,X_test,y_train,y_test = train_test_split(X,y, test_size = 0.3,random_state = 42)


# Scaling the data
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


# All models
clf1 = LogisticRegression(random_state= 42,max_iter = 1000)
clf2 = SVC(probability=True)
clf3 = DecisionTreeClassifier(random_state=42)
clf4 = GradientBoostingClassifier(random_state=42)

# Voting Classification
voting = VotingClassifier(estimators =[
    ('l2',clf1),
    ('svc',clf2),
    ('dt',clf3),
    ('gb', clf4) 
],voting = 'soft',weights=[2, 1, 1,1])


# Parameter Tuning
param_grid = {
    'l2__C': [0.1, 1, 10],                  
    'svc__C': [0.1, 1, 10],               
    'svc__kernel': ['linear', 'rbf'],       
    'dt__max_depth': [3, 5, 7],             
    'dt__min_samples_split': [2, 5, 10],
    'gb__n_estimators': [50, 100, 200],
    'gb__learning_rate': [0.01, 0.1, 0.2]
}

grid_search = GridSearchCV(estimator = voting, param_grid = param_grid , cv = 5, scoring= 'accuracy')

grid_search.fit(X_train_scaled,y_train)

best_model = grid_search.best_estimator_

y_pred = best_model.predict(X_test)

c:\Users\rajes\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\ma\core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,
c:\Users\rajes\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
c:\Users\rajes\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but SVC was fitted without feature names
  warnings.warn(
c:\Users\rajes\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(
c:\Users\rajes\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but GradientBoostingClassifier was fitted without feature names
  warnings.warn(


In [12]:
from sklearn.model_selection import cross_val_score

val_score = cross_val_score(best_model,X_train_scaled,y_train,cv = 5)

val_score.mean()

0.7931983385254412

# Model Evaluation

In [10]:
from sklearn.metrics import accuracy_score,f1_score,precision_score, confusion_matrix,recall_score

acc = accuracy_score(y_test,y_pred)
f1 = f1_score(y_test,y_pred)
prec_score = precision_score(y_test,y_pred)
recall = recall_score(y_test,y_pred)
conf_martix = confusion_matrix(y_test,y_pred)


print(f"accuracy_score  : {acc}")
print(f"f1_score        : {f1}")
print(f"precision_score : {prec_score}")
print(f"recall score    : {recall}")
print(f"\n Confusion Matrix : \n {conf_martix}")

comparing  = pd.DataFrame({
    'Orginal_values ' : y_test,
    'Predicted_values' : y_pred
})

#Accurcy percentage :

print(f"\n Comparing : \n{comparing.tail()}")

percentage = acc * 100
print(f" \nAccuracy percecntage {percentage :.2f}%")

accuracy_score  : 0.3463203463203463
f1_score        : 0.5144694533762058
precision_score : 0.3463203463203463
recall score    : 1.0

 Confusion Matrix : 
 [[  0 151]
 [  0  80]]

 Comparing : 
     Orginal_values   Predicted_values
619                1                 1
198                1                 1
538                0                 1
329                0                 1
302                0                 1
 
Accuracy percecntage 34.63%
